In [1]:
import sigpde.torch as sig
import torch
import math
device = torch.device('cuda:0')

In [23]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 3)
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 3)

In [3]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device) / math.sqrt(length)
  start = torch.zeros([batch_size, 1, dimension], device=device, dtype=torch.double)
  random_walks = torch.cat((start, random_walks), dim=1)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

def add_time(x, start=0, stop=1):
    device = x.device
    dtype = x.dtype

    l = x.shape[1]

    t = torch.linspace(start, stop, l, device=device, dtype=dtype)
    t = t.unsqueeze(0).unsqueeze(-1)
    return torch.cat((x, t.expand(x.shape[0], x.shape[1], 1)), dim=-1)

def time_norm(x, time=True):
    if time:
        x = add_time(x)
    return x / x.pow(2).sum(dim=2).sqrt().max(dim=1).values.view(x.shape[0], 1, 1)

In [56]:
x = generate(4, 512, 50, device=device)
y = generate(4, 512, 50, device=device)

In [53]:
x = time_norm(x)
y = time_norm(y)

In [54]:
kernel.pairwise(x)

tensor([2.4937, 2.6119, 2.5805, 2.4962], device='cuda:0', dtype=torch.float64)

In [62]:
kernel2.pairwise(x, y, bisections=7, nr_iterations=0)

tensor([1.0264, 0.8751, 1.0394, 1.2287], device='cuda:0', dtype=torch.float64)